# Visualise *Ergo* data
This notebook contains various ad-hoc means of visualising data collected from *Ergo*. See the report [here](https://git.cs.sun.ac.za/Computer-Science/rw771/2022/26723077-TG7-doc) or the source code [here](https://git.cs.sun.ac.za/Computer-Science/rw771/2022/26723077-TG7-src).

## Import libraries and define constants

In [ ]:
%load_ext autoreload
%autoreload 2
from common_utils import *
gesture_info = get_gesture_info()
dir_files = get_dir_files()

## Interactively view the raw data
This cell lets you view the raw training data as found in `../gesture_data/train/{gesture}/{timestamp}`. Run the cell, and then you can choose which gesture to view (ie `gesture0001`, `gesture0002`, `gesture0003`) and then which observation of that gesture to view (ie `1`, `2`, `3`). 

Note that the raw sensor data is not all centered at the same position (so a no-acceleration value from one sensor might be different to the no-acceleration value from another sensor). Check the `normalise` checkbox in order to roughly center all the values around their common average. Specifically, the `-z` sensors are usually quite different from the `-x` and `-y` sensors since the `-z` component is in the direction of gravity.

In [ ]:
max_files = max([len(files) for files in dir_files.values()])
dirs = sorted(list(dir_files.keys()))

@interact(gesture=dirs, observation_index=(0, max_files, 1))
def showfile(gesture, observation_index):
    try:
        filenames = sorted([f for f in os.listdir(f'../gesture_data/train/{gesture}') if f.endswith('.txt')])
        idx = min(observation_index, len(filenames)-1)
        if idx == len(filenames)-1:
            print(f"""Warning: you've selected an index ({idx}) greater than the number of observations for gesture {gesture} ({len(filenames)}) so observation {len(filenames)-1} is being shown instead.""")
        filename = f'../gesture_data/train/{gesture}/{filenames[idx]}'
        arr = np.genfromtxt(filename, delimiter=",")[:,1:]
    except (FileNotFoundError, IndexError):
        print(f"Failed to find file for {gesture=}, {idx=}, {filename=}")
        return
    df = read_to_df(filename, normalise=True)
    plot_raw_gesture(df, filename)
    print(df.min().min(), df.max().max())


## (Work in progress) Live view the most recent data

In [ ]:
last_most_recent_file = ''
while True:
    all_files = []
    for gesture, files in get_dir_files().items():
        all_files.extend([f'{f},{gesture}' for f in files])

    file, gesture = sorted(all_files)[-1].split(',')
    most_recent_file = f'../gesture_data/train/{gesture}/{file}'
    if last_most_recent_file != most_recent_file:
        last_most_recent_file = most_recent_file
        arr = np.genfromtxt(most_recent_file, delimiter=",")[:,1:]
        df = read_to_df(most_recent_file, normalise=True)
        fig, ax = plot_raw_gesture(df, most_recent_file)
        print(f'plotting {most_recent_file}')
        plt.show()
